### Transformers y aprendizaje por transferencia 

**Usando fastai**

In [ ]:
from fastai.text.all import *

In [ ]:
ruta = untar_data(URLs.IMDB)
ruta.ls()

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [ ]:
dls.show_batch()

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
learn.show_results()

In [ ]:
learn.predict("That movie was cool!")

#### ULMFiT

ULMFiT (Universal Language Model Fine-tuning) es un enfoque innovador para el aprendizaje por transferencia desarrollado por Jeremy Howard y Sebastian Ruder de fast.ai en 2018. Este modelo ha sido uno de los primeros en demostrar que es posible usar el aprendizaje de transferencia de manera efectiva en tareas de NLP, algo que antes se creía que era casi exclusivo de las áreas de visión por computadora.

ULMFiT se basa en varios principios y técnicas clave para adaptar los modelos de lenguaje pre-entrenados a nuevas tareas con relativamente poca data y sin requerir un entrenamiento extensivo desde cero. Los principales componentes de ULMFiT incluyen:

- Modelo de lenguaje preentrenado: Comienza con un modelo de lenguaje preentrenado (generalmente un LSTM con técnicas de regularización aplicadas) sobre un corpus grande (como el dataset de Wikipedia). Este modelo aprende una representación general del lenguaje.
- Ajuste discriminativo de la tasa de aprendizaje ("Discriminative Fine-tuning"): Desde el principio, cada capa del modelo aprende a diferentes velocidades, ajustando así diferentes partes del modelo en diferentes tasas. Por ejemplo, las capas más bajas pueden necesitar menos ajustes que las capas superiores. ULMFiT aplica diferentes tasas de aprendizaje a diferentes capas del modelo durante el afinamiento.

- Descongelación gradual de capas ("gradual unfreezing"): Para evitar que el aprendizaje rápido destruya las representaciones aprendidas del modelo preentrenado, ULMFiT descongela las capas del modelo gradualmente. Comienza por afinar solo las últimas capas y progresivamente va descongelando las anteriores, permitiendo que todo el modelo se ajuste de manera más efectiva.
- Concatenación de técnicas de regularización ("concat pooling"): ULMFiT no solo utiliza el estado oculto final de los modelos recurrentes para la clasificación de textos o otras tareas. En lugar de eso, concatena el estado oculto máximo y promedio de todos los tiempos junto con el estado final, lo que proporciona una buena  representación del texto completo.

Referencia:[ULMFiT](https://docs.fast.ai/examples/ulmfit.html)

In [ ]:
dls_lm = TextDataLoaders.from_folder(ruta, is_lm=True, valid_pct=0.1)

In [ ]:
dls_lm.show_batch(max_n=5)

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()],
    path=ruta, wd=0.1).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXTO = "I liked this movie because"
N_PALABRAS = 40
N_ORACIONES = 2
preds = [learn.predict(TEXTO, N_PALABRAS, temperature=0.75) 
         for _ in range(N_ORACIONES)]

In [ ]:
print("\n".join(preds))

#### Entrenamiento de un clasificador de texto

In [ ]:
dls_clas = TextDataLoaders.from_folder(
    untar_data(URLs.IMDB), valid='test',
    text_vocab=dls_lm.vocab)

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

Algunas de las cosas que puede hacer con la clasificación de texto incluyen: 

* Predecir el lenguaje de programación de algún código fuente 

* Creación de un clasificador de spam de correo electrónico simple 

* Mejorar la funcionalidad de un bot de moderación de contenido automatizado para chats o foros en línea 

* Categorización de documentos en función del lenguaje.


**Inferencia con Hugging Face**

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

texto = "With great power comes great "
tokens_indexado = tokenizer.encode(texto)

tokens_tensor = torch.tensor([tokens_indexado])

In [ ]:
print(tokens_tensor)

Ahora, hagamos la inferencia real, que es, nuevamente, solo unas pocas líneas de código gracias a la librería de transformadores `Huggingface`.

In [ ]:
modelo = GPT2LMHeadModel.from_pretrained('gpt2')

modelo.eval()

with torch.no_grad():
    salidas = modelo(tokens_tensor)
    predicciones = salidas[0]

indice_predicho= torch.argmax(predicciones[0, -1, :]).item()
texto_predicho = tokenizer.decode(tokens_indexado + [indice_predicho])
print(texto_predicho)

**Cargando modelos**

In [ ]:
modelo = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
modelo.eval()

**Generamos predicciones**

In [ ]:
with torch.no_grad():
    salidas = modelo(tokens_tensor)
    predicciones = salidas[0]

In [ ]:
salidas[0].shape

Dado que parece que `salidas[0]` es lo que queremos, lo asignaremos a la variable `predicciones`. Colocamos todo juntos y los envolvemos en el contexto `torch.no_grad()` nos da ese mini bloque de código que teníamos antes: 

In [ ]:
with torch.no_grad():
    salidas = modelo(tokens_tensor)
    predicciones = salidas[0]

In [ ]:
indice_predicho= torch.argmax(predicciones[0, -1, :]).item()

In [ ]:
texto_predicho = tokenizer.decode(tokens_indexado + [indice_predicho])
print(texto_predicho)

**Ejercicio**

Completa las dos primeras secciones del curso de [HuggingFaces](https://huggingface.co/learn/nlp-course/es/chapter1/1).

In [ ]:
# Tu respuesta.